In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import expr, col, column
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
companies = spark.read.json("hdfs://namenode:9000/project20221/raw/VietstockFinance_companies.json")
stockInfo = spark.read.option("header","true").csv("hdfs://namenode:9000/project20221/raw/ssiDataApi.csv")
news = spark.read.json("hdfs://namenode:9000/project20221/raw/analysNews.json")

# companies.select(expr("Mã CK AS stock_code"))

companies.show(1, False)
stockInfo.show(1, False)
news.show(1, False)


In [ ]:
fullInfo = companies.join(stockInfo, companies.stock_code == stockInfo.stockSymbol, "inner")
fullInfo.show(truncate=False)

In [45]:
from pyspark.sql.functions import col

stock_code = 'BAF'
news.filter(col("tagCodes").like(stock_code)).show()

+-------+------+--------------------+--------------------+----------+------------+--------+----------+--------+--------------------+------------+--------------------+--------+--------------------+
|hotNews|locale|        newsAbstract|         newsContent|  newsDate|   newsGroup|  newsId|newsSource|newsTime|           newsTitle|    newsType|             newsUrl|tagCodes|        thumbnailUrl|
+-------+------+--------------------+--------------------+----------+------------+--------+----------+--------+--------------------+------------+--------------------+--------+--------------------+
|      N|    VN|Thị giá BAF đang ...|<head>\n</HEAD>\n...|2022-12-21|company_news|10272215|     CAFEF|17:52:17|Công ty của Chủ t...|company_news|https://cafef.vn/...|     BAF|https://cafefcdn....|
|      N|    VN|Thị giá BAF đang ...|<head>\n</HEAD>\n...|2022-12-21|company_news|10272215|     CAFEF|17:52:17|Công ty của Chủ t...|company_news|https://cafef.vn/...|     BAF|https://cafefcdn....|
|      N|    VN